In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ospkg.constants import RESULTS_DIR
from ospkg.ensemble_snmmi import aggregate_results, build_dataframe, compute_c_indices, ensemble
from ospkg.utils import apply_alpha_correction, compute_alpha

# TASK 1: MEAN SQUARED ERROR BETWEEN PREDICTED AND OBSERVED PFS IN MONTHS.

## 1.1 SELECTION OF MODELS: ENSEMBLE OF BEST PERFORMING MODELS.

In this section,

- We rank the models according to their average MSE over the 5 validation folds. 
- We create ensembles of increasing number of models by combining the TOP_1 (single models), TOP_2, TOP_3, ..., TOP_N best performing models (i.e., models with lowest MSE). 
- For each ensemble, we compute the ensemble's prediction (estimated PFS) by averaging the predictions of each of the model members.
- We calculate the MSE for each ensemble, and select the ensemble with the lowest MSE.

In [ ]:
criterion = "mse-avg"
df_val = build_dataframe(data_dir=RESULTS_DIR, criterion=criterion)
df_val.to_csv("tmp_val.csv")

In [ ]:
# Perform the search for best ensemble for the first task of the challenge.
# We've noticed that performance does not change much after we add 10+ models. So we stop at 16.

TOP_N = 25
aggregation_method = "mean"  # mean, median, no-tail

# Parameters for plot
k = int(np.sqrt(TOP_N))
plt.figure(figsize=(k * 4, k * 4))

# Loop and create increasing number of ensemble models!
best_ensemble_n = 1  # TOP_1 initially.
best_MSE = np.inf

for n_ in range(1, TOP_N + 1):

    df_ensemble_mse = ensemble(df=df_val, top_n=n_, criterion="mse-avg", fold=-1)

    pred, true, event = aggregate_results(df=df_ensemble_mse, method=aggregation_method)

    error = pred - true
    mse = np.mean(error**2)

    plt.subplot(k, k, n_)

    # Get Errors for Censored vs Uncensored
    err_censored_hist, bins = np.histogram(error[event == 0.0], bins=np.arange(-100, 100, 4))
    plt.step(bins[1:], err_censored_hist, "r-", linewidth=1, label="Censored")

    err_censored_hist, bins = np.histogram(error[event == 1.0], bins=np.arange(-100, 100, 4))
    plt.step(bins[1:], err_censored_hist, "g-", linewidth=1, label="Uncensored")

    plt.title(f"TOP_{n_} - MSE = {mse: 1.0f}")

    if n_ > k * (k - 1):
        plt.xlabel("(PRED - TRUE) [Months]")
    if (n_ - 1) % k == 0:
        plt.ylabel("Counts")

    if n_ == 1:
        plt.legend()

    # UPDATE BEST ENSEMBLE FOUND!
    if mse < best_MSE:
        best_ensemble_n = n_
        best_MSE = mse

plt.savefig(f"Ensemble_top_N_{aggregation_method}_for_{criterion}.png")

In [ ]:
# The best ensemble:
print(
    f"The best ensemble found for {criterion} consists of the top {best_ensemble_n} models, and yields an average MSE = {best_MSE: 1.1f} months ** 2"
)

best_ensemble_df = ensemble(
    df=df_val, top_n=best_ensemble_n, criterion=criterion, fold=-1, verbose=True
)

In [ ]:
pred, true, event = aggregate_results(df=best_ensemble_df, method=aggregation_method)
error = pred - true
plt.plot(pred[event == 0], error[event == 0], "ro", label="Censored")
plt.plot(pred[event == 1], error[event == 1], "go", label="Uncensored")
plt.title(
    f"Ensemble: TOP {best_ensemble_n}; \n criterion: {criterion}; aggregation: {aggregation_method}"
)
plt.xlabel("Predicted Time to Event [Months]")
plt.ylabel("(PRED - TRUE)")
plt.legend()

In [ ]:
pred_opt, th, al = compute_alpha(pred, true)

print(th, al, ((pred - true) ** 2).sum() / len(pred), ((pred_opt - true) ** 2).sum() / len(pred))

error = pred_opt - true
plt.plot(pred_opt[event == 0], error[event == 0], "ro", label="Censored")
plt.plot(pred_opt[event == 1], error[event == 1], "go", label="Uncensored")
plt.title(
    f"Ensemble: TOP {best_ensemble_n} with ALPHA correction; \n criterion: {criterion}; aggregation: {aggregation_method}"
)
plt.xlabel("Predicted Time to Event [Months]")
plt.ylabel("(PRED - TRUE)")
plt.legend()

## >>> Generate Results over TEST set, for TASK 1

In [ ]:
# Build Test Set Dataframe
df_test = build_dataframe(data_dir=RESULTS_DIR, test=True, criterion="mse-avg")

# Reminde me of the selected Ensemble:
print("The Selected Ensemble is composed of:")
for selected_ in best_ensemble_df["Model"].unique():
    print(f" >> {selected_}")

# Retrieve these models from TEST set dataframe
df_test_selected = df_test[df_test["Model"].isin(best_ensemble_df["Model"].unique())]

if (best_ensemble_df["Model"].unique() != df_test_selected["Model"].unique()).all():
    raise AssertionError("Couldn't find all the models in the TEST set dataframe")

In [ ]:
# Aggregate the results:
if aggregation_method == "mean":
    pred_continuous_pfs = df_test_selected.groupby(by="PatientID")["PFS_pred"].mean()
else:
    raise ValueError("Are you sure? Mean aggregation seemed to be the best so far. PLease review")

# Apply correction:
pred_submit = pd.DataFrame(pred_continuous_pfs)
pred_submit["Predicted PFS"] = pred_submit.apply(
    lambda x: apply_alpha_correction(x["PFS_pred"], th, al), axis=1
)

pred_submit = pred_submit.drop(labels="PFS_pred", axis=1)
pred_submit.index = pred_submit.index.astype(int)
pred_submit = pred_submit.sort_values(by="PatientID", ascending=True)
pred_submit.to_csv("PFSContinuousOutcome.csv")

# And plot the distributions and compare against Validation
hist_val, bins_ = np.histogram(pred_opt, bins=30)
hist_test, _ = np.histogram(pred_submit["Predicted PFS"], bins=bins_)

plt.step(bins_[1:], hist_val / np.sum(hist_val), "r", label="Validation")
plt.step(bins_[1:], hist_test / np.sum(hist_test), "b", label="Test")
plt.title("TASK 1")
plt.xlabel("Estimated Time to Event [Months]")
plt.ylabel("Normalized Counts")
plt.legend()

## 1.2 SELECTION OF MODELS: ENSEMBLE OF RANDOM MODELS. [DISABLED FOR NOW AS WE DO NOT WANT TO OVERFIT]

In this section,

- We create ensembles by sampling up to 5 models, at random, several times. 
- For each ensemble, we calculate MSE.
- We select the ensemble with the lowest MSE.

In [ ]:
# assert False, "We are NOT running this. Very high chance of overfitting."

num_samples = 10000
TOP_N = 0

top_1 = ensemble(df=df_val, top_n=1, criterion=criterion, fold=-1)  # TOP_1 initially.
best_ensemble_df = top_1
best_MSE = best_ensemble_df["mse-avg"].mean()

print(f"Current MSE to beat: {best_MSE: 1.1f}")

for i in range(num_samples):

    num_models = np.random.choice([1, 2, 3, 4], 1)
    df_ensemble = ensemble(
        df=df_val, top_n=TOP_N, criterion=criterion, fold=-1, num_random=num_models[0]
    )

    # Always include Top_1?
    df_ensemble = pd.concat([top_1, df_ensemble])

    pred, true, event = aggregate_results(df=df_ensemble, method=aggregation_method)

    mse = np.mean((pred - true) ** 2)

    if mse < best_MSE:
        print(f"Found lower MSE: {mse: 1.1f}, sample = {i} /{num_samples}")
        best_ensemble_df = df_ensemble.copy()
        best_MSE = mse

# The best ensemble:
print(
    f"The best ensemble found for {criterion} consists of the {best_ensemble_df['Model'].unique()} models, and yields an average MSE = {best_MSE: 1.1f} months ** 2"
)

pred, true, event = aggregate_results(df=best_ensemble_df, method=aggregation_method)
error = pred - true
plt.plot(pred[event == 0], error[event == 0], "ro", label="Censored")
plt.plot(pred[event == 1], error[event == 1], "go", label="Uncensored")
plt.title(
    f"Ensemble: Best Random Models; \n criterion: {criterion}; aggregation: {aggregation_method}"
)
plt.xlabel("Predicted Time to Event [Months]")
plt.ylabel("(PRED - TRUE)")
plt.legend()

### Apply alpha correction
For non-mse error (most survival methods) we correct the estimates of pfs because larger PFS tend to be censored and by this the 
actual estimate will tend to overestimate the actual pfs.  

In [ ]:
pred_opt, th, al = compute_alpha(pred, true)

print(
    th, al, ((pred - true) ** 2).sum() / len(pred), ((pred_opt - true) ** 2).sum() / len(pred_opt)
)

error = pred_opt - true
plt.plot(pred_opt[event == 0], error[event == 0], "ro", label="Censored")
plt.plot(pred_opt[event == 1], error[event == 1], "go", label="Uncensored")
plt.title(
    f"Ensemble: Best Random Models; \n criterion: {criterion}; aggregation: {aggregation_method}"
)
plt.xlabel("Predicted Time to Event [Months]")
plt.ylabel("(PRED - TRUE)")
plt.legend()

# >>> GENERATE RESULTS OVER TEST, FOR TASK 1: RISKY

In [ ]:
# Build Test Set Dataframe
df_test = build_dataframe(data_dir=RESULTS_DIR, test=True, criterion="mse-avg")

# Reminde me of the selected Ensemble:
print("The Selected Ensemble is composed of:")
for selected_ in best_ensemble_df["Model"].unique():
    print(f" >> {selected_}")

# Retrieve these models from TEST set dataframe
df_test_selected = df_test[df_test["Model"].isin(best_ensemble_df["Model"].unique())]

if (best_ensemble_df["Model"].unique() != df_test_selected["Model"].unique()).all():
    raise AssertionError("Couldn't find all the models in the TEST set dataframe")

In [ ]:
# Aggregate the results:
if aggregation_method == "mean":
    pred_continuous_pfs = df_test_selected.groupby(by="PatientID")["PFS_pred"].mean()
else:
    raise ValueError("Are you sure? Mean aggregation seemed to be the best so far. PLease review")

# Apply correction:
pred_submit = pd.DataFrame(pred_continuous_pfs)
pred_submit["Predicted PFS"] = pred_submit.apply(
    lambda x: apply_alpha_correction(x["PFS_pred"], th, al), axis=1
)

pred_submit = pred_submit.drop(labels="PFS_pred", axis=1)
pred_submit.index = pred_submit.index.astype(int)
pred_submit = pred_submit.sort_values(by="PatientID", ascending=True)
pred_submit.to_csv("PFSContinuousOutcomeRISKY.csv")

# And plot the distributions and compare against Validation
hist_val, bins_ = np.histogram(pred_opt, bins=30)
hist_test, _ = np.histogram(pred_submit["Predicted PFS"], bins=bins_)

plt.step(bins_[1:], hist_val / np.sum(hist_val), "r", label="Validation")
plt.step(bins_[1:], hist_test / np.sum(hist_test), "b", label="Test")
plt.title("TASK 1: RISKY")
plt.xlabel("Estimated Time to Event [Months]")
plt.ylabel("Normalized Counts")
plt.legend()

# TASK 2: AVERAGE C-INDEX FOR 1, 2 and 3 YEARS PFS TO ASSESS THE BEST CLASSIFLYING MODEL.

## 2.1 SELECTION OF MODELS: ENSEMBLE OF TOP_N PERFORMING MODELS.

In [ ]:
# Reload DataFrame, exclude models with invalid outputs for c-index.
criterion = "c-index-123-avg"
df_val = build_dataframe(data_dir=RESULTS_DIR, criterion=criterion)

In [ ]:
# Perform the search.
criterion = "c-index-123-avg"
TOP_N = 40
aggregation_method = "mean"

best_c = 0
best_ensemble_df = pd.DataFrame()

for n_ in range(1, TOP_N + 1):

    df_ensemble_c123 = ensemble(df=df_val, top_n=n_, criterion=criterion, fold=-1)

    prob_1, true_1, event_1 = aggregate_results(
        df=df_ensemble_c123, method=aggregation_method, task="SurvProb1"
    )
    prob_2, true_2, event_2 = aggregate_results(
        df=df_ensemble_c123, method=aggregation_method, task="SurvProb2"
    )
    prob_3, true_3, event_3 = aggregate_results(
        df=df_ensemble_c123, method=aggregation_method, task="SurvProb3"
    )

    # Sanity checks
    assert (true_1 == true_2).all()
    assert (event_1 == event_2).all()
    assert (true_1 == true_3).all()
    assert (event_1 == event_3).all()

    # Compute C-indices
    c_1, c_2, c_3 = compute_c_indices(
        dura=true_1.to_list(),
        ev=event_1.to_list(),
        p1=prob_1.to_list(),
        p2=prob_2.to_list(),
        p3=prob_3.to_list(),
    )

    c_123 = (c_1 + c_2 + c_3) / 3

    if c_123 > best_c:
        best_c = c_123
        best_ensemble_df = df_ensemble_c123.copy()

# The best ensemble:
print(
    f"The best ensemble found for {criterion} consists of the {best_ensemble_df['Model'].unique()} models, and yields an average C-123 index = {best_c}"
)

## >>> GENERATE RESULTS OVER TEST FOR TASK 2

In [ ]:
# Build Test Set Dataframe
df_test = build_dataframe(data_dir=RESULTS_DIR, test=True, criterion="c-index-123-avg")

# Reminde me of the selected Ensemble:
print("The Selected Ensemble is composed of:")
for selected_ in best_ensemble_df["Model"].unique():
    print(f" >> {selected_}")

# Retrieve these models from TEST set dataframe
df_test_selected = df_test[df_test["Model"].isin(best_ensemble_df["Model"].unique())]

if (best_ensemble_df["Model"].unique() != df_test_selected["Model"].unique()).all():
    raise AssertionError("Couldn't find all the models in the TEST set dataframe")

In [ ]:
# Aggregate the results:
if aggregation_method == "mean":
    pred_prob_1 = df_test_selected.groupby(by="PatientID")["SurvProb1"].mean()
    pred_prob_2 = df_test_selected.groupby(by="PatientID")["SurvProb2"].mean()
    pred_prob_3 = df_test_selected.groupby(by="PatientID")["SurvProb3"].mean()
else:
    raise ValueError("Are you sure? Mean aggregation seemed to be the best so far. PLease review")

pred_submit = pd.DataFrame(
    {
        "Probability for 1 year": pred_prob_1,
        "Probability for 2 year": pred_prob_2,
        "Probability for 3 year": pred_prob_3,
    }
)

pred_submit.index = pred_submit.index.astype(int)
pred_submit = pred_submit.sort_values(by="PatientID", ascending=True)
pred_submit.to_csv("ClassificationOutcome.csv")

# Plot the distributions
prob_1, true_1, event_1 = aggregate_results(
    df=best_ensemble_df, method=aggregation_method, task="SurvProb1"
)
prob_2, true_2, event_2 = aggregate_results(
    df=best_ensemble_df, method=aggregation_method, task="SurvProb2"
)
prob_3, true_3, event_3 = aggregate_results(
    df=best_ensemble_df, method=aggregation_method, task="SurvProb3"
)

val_probs = [prob_1, prob_2, prob_3]
test_probs = [pred_prob_1, pred_prob_2, pred_prob_3]

plt.figure(figsize=(18, 5))
bins_ = np.arange(0, 1, 0.02)
for i in range(3):
    plt.subplot(1, 3, i + 1)

    hist_val, _ = np.histogram(val_probs[i], bins=bins_)
    hist_test, _ = np.histogram(test_probs[i], bins=bins_)

    plt.step(bins_[1:], hist_val / np.sum(hist_val), "r", label="Validation")
    plt.step(bins_[1:], hist_test / np.sum(hist_test), "b", label="Test")
    plt.title(f"TASK 2: Probability {i + 1} year")
    plt.xlabel("Probability")
    plt.ylabel("Normalized Counts")
    plt.xlim([0.5, 1])
    plt.legend()

## 2.2 SELECTION OF MODELS: ENSEMBLE OF N RANDOM MODELS.

In [ ]:
num_samples = 10000
TOP_N = 0

best_c = 0
best_ensemble_df = pd.DataFrame()

for i in range(num_samples):

    num_models = np.random.choice([1, 2, 3, 4], 1)
    df_ensemble_c123 = ensemble(
        df=df_val, top_n=TOP_N, criterion=criterion, fold=-1, num_random=num_models[0]
    )

    prob_1, true_1, event_1 = aggregate_results(
        df=df_ensemble_c123, method=aggregation_method, task="SurvProb1"
    )
    prob_2, true_2, event_2 = aggregate_results(
        df=df_ensemble_c123, method=aggregation_method, task="SurvProb2"
    )
    prob_3, true_3, event_3 = aggregate_results(
        df=df_ensemble_c123, method=aggregation_method, task="SurvProb3"
    )

    # Sanity checks
    assert (true_1 == true_2).all()
    assert (event_1 == event_2).all()
    assert (true_1 == true_3).all()
    assert (event_1 == event_3).all()

    # Compute C-indices
    c_1, c_2, c_3 = compute_c_indices(
        dura=true_1.to_list(),
        ev=event_1.to_list(),
        p1=prob_1.to_list(),
        p2=prob_2.to_list(),
        p3=prob_3.to_list(),
    )

    c_123 = (c_1 + c_2 + c_3) / 3

    if c_123 > best_c:
        print(f"Found higher C-Index-123: {c_123: 1.3f}, sample = {i}/{num_samples}")
        best_c = c_123
        best_ensemble_df = df_ensemble_c123.copy()

# The best ensemble:
print(
    f"The best ensemble found for {criterion} consists of the {best_ensemble_df['Model'].unique()} models, and yields an average C-123 index = {best_c}"
)

## >>> GENERATE RESULTS OVER TEST, FOR TASK 2: RISKY

In [ ]:
# Build Test Set Dataframe
df_test = build_dataframe(data_dir=RESULTS_DIR, test=True, criterion="c-index-123-avg")

# Remind me of the selected Ensemble:
print("The Selected Ensemble is composed of:")
for selected_ in best_ensemble_df["Model"].unique():
    print(f" >> {selected_}")

# Retrieve these models from TEST set dataframe
df_test_selected = df_test[df_test["Model"].isin(best_ensemble_df["Model"].unique())]

if (best_ensemble_df["Model"].unique() != df_test_selected["Model"].unique()).all():
    raise AssertionError("Couldn't find all the models in the TEST set dataframe")

In [ ]:
# Aggregate the results:
if aggregation_method == "mean":
    pred_prob_1 = df_test_selected.groupby(by="PatientID")["SurvProb1"].mean()
    pred_prob_2 = df_test_selected.groupby(by="PatientID")["SurvProb2"].mean()
    pred_prob_3 = df_test_selected.groupby(by="PatientID")["SurvProb3"].mean()
else:
    raise ValueError("Are you sure? Mean aggregation seemed to be the best so far. PLease review")

pred_submit = pd.DataFrame(
    {
        "Probability for 1 year": pred_prob_1,
        "Probability for 2 year": pred_prob_2,
        "Probability for 3 year": pred_prob_3,
    }
)

pred_submit.index = pred_submit.index.astype(int)
pred_submit = pred_submit.sort_values(by="PatientID", ascending=True)
pred_submit.to_csv("ClassificationOutcomeRISKY.csv")

# Plot the distributions
prob_1, true_1, event_1 = aggregate_results(
    df=best_ensemble_df, method=aggregation_method, task="SurvProb1"
)
prob_2, true_2, event_2 = aggregate_results(
    df=best_ensemble_df, method=aggregation_method, task="SurvProb2"
)
prob_3, true_3, event_3 = aggregate_results(
    df=best_ensemble_df, method=aggregation_method, task="SurvProb3"
)

val_probs = [prob_1, prob_2, prob_3]
test_probs = [pred_prob_1, pred_prob_2, pred_prob_3]

plt.figure(figsize=(18, 5))
bins_ = np.arange(0, 1, 0.02)
for i in range(3):
    plt.subplot(1, 3, i + 1)

    hist_val, _ = np.histogram(val_probs[i], bins=bins_)
    hist_test, _ = np.histogram(test_probs[i], bins=bins_)

    plt.step(bins_[1:], hist_val / np.sum(hist_val), "r", label="Validation")
    plt.step(bins_[1:], hist_test / np.sum(hist_test), "b", label="Test")
    plt.title(f"TASK 2: Probability {i + 1} year - RISKY")
    plt.xlabel("Probability")
    plt.ylabel("Normalized Counts")
    plt.xlim([0.4, 1])
    plt.legend()